In [86]:
import json
import requests
import urllib.parse
base_url = 'https://itunes.apple.com/search'

artist = 'Nirvana'

params = {
    'term': artist,
    'limit': 112,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [87]:
# Все доступные ключи объекта
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [88]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [89]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

In [90]:
print(set(map(lambda x: x['artistName'], data)))

{'Nirvana'}


In [91]:
# Фильтрация с целью оставить искомого артиста
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)
release_date_extractor = lambda x: x['releaseDate']

first_song = min(data, key=release_date_extractor)

print('First song:', first_song['trackName'], first_song['releaseDate'])

latest_song = max(data, key=release_date_extractor)

print('Last song:', latest_song['trackName'], latest_song['releaseDate'])

First song: About a Girl 1989-06-15 07:00:00+00:00
Last song: All Apologies (Live & Loud) 2013-09-23 12:00:00+00:00


In [92]:
Count = lambda x: x['trackName']
Count_Of_Songs = len(list(map(Count, data)))
print('Count of songs:' ,Count_Of_Songs)

Count of songs: 112


In [93]:
Genre = lambda x: x['primaryGenreName']
Popular_Genre = (list((map(Genre, data))))
if Popular_Genre.count('Grunge') > Popular_Genre.count('Alternative'):
    print('Most popular genre:','Grunge -', Popular_Genre.count('Grunge'))
if Popular_Genre.count('Rock') > Popular_Genre.count('Alternative'):
    print('Most popular genre:','Rock -', Popular_Genre.count('Rock'))
else:
    print('Most popular genre:','Alternative -', Popular_Genre.count('Alternative'))

Most popular genre: Alternative - 104


In [94]:
Album = lambda x: x['collectionId']
Album_count = len(list(set(map(Album, data))))
print('Album count:', Album_count)

Album count: 11


In [95]:
time_extractor = lambda x: x['trackTimeMillis']

Duration_of_songs = sum(list(map(time_extractor, data)))
print('Duration of songs:', convert_millis(Duration_of_songs))

Average_duration = Duration_of_songs / Count_Of_Songs
print('Average duration of songs:', convert_millis(Average_duration))

Median_time = list(map(time_extractor, data))
import statistics
print('Mediana of songs:', convert_millis(statistics.median(Median_time)))

longest_song = max(data, key=time_extractor)
print('Longest song:', longest_song['trackName'], convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)
print('Shortest song:', shortest_song['trackName'], convert_millis(shortest_song['trackTimeMillis']))

Duration of songs: 6H24M4S
Average duration of songs: 0.0H3.0M25.0S
Mediana of songs: 0.0H3.0M32.0S
Longest song: Endless, Nameless 0H6M42S
Shortest song: Intro (Live at Pine Street Theatre) 0H0M52S


In [96]:
Song_cost = lambda x: x['trackPrice']

All_price = sum(list(map(Song_cost, data)))
print('Price of all songs:', All_price, '$')

Average_price = All_price / Count_Of_Songs
print('Average price of songs:', Average_price, '$')

Median_cost = list(map(Song_cost, data))
import statistics
print('Median of songs cost:', statistics.median(Median_cost), '$')

Expensive_song = max(data, key=Song_cost)
print('Expensive song:', Expensive_song['trackName'], Expensive_song['trackPrice'],'$')

Cheap_song = min(data, key=Song_cost)
if Cheap_song['trackPrice'] == -1 : 
    print('Cheap song:', Cheap_song['trackName'],'Free')
else:
    print('Cheap song:', Cheap_song['trackName'], Cheap_song['trackPrice'], '$')

Price of all songs: 134.27999999999997 $
Average price of songs: 1.1989285714285711 $
Median of songs cost: 1.29 $
Expensive song: Smells Like Teen Spirit 1.29 $
Cheap song: About a Girl 0.99 $


In [98]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']
del sorted_by_album

print(album_names)

{28436515: 'Lithium (Acoustic Version) - Single', 669728060: 'Bleach', 1413920644: 'Nirvana', 1440778637: 'Sliver: The Best of the Box', 1440783617: 'Nevermind', 1440797324: 'Live at Reading', 1440830905: 'In Utero', 1440833316: 'With the Lights Out', 1440858699: 'In Utero (20th Anniversary Edition)', 1440892370: 'MTV Unplugged In New York (Live Acoustic)', 1445283308: 'Lithium - Single'}


In [101]:
Album_name = album_names[1440797324]
print('Album name:', Album_name)

Album name: Live at Reading


In [102]:
Album_value = list(album_groups[1440797324])

Count = lambda x: x['trackId']
Count_Of_Album_Songs = len(list(set(map(Count, Album_value))))
print('Count of songs:', Count_Of_Album_Songs)

Count of songs: 24


In [104]:
Genre = lambda x: x['primaryGenreName']
Popular_Genre = (list((map(Genre, Album_value))))
if Popular_Genre.count('Grunge') > Popular_Genre.count('Alternative'):
    print('Most popular genre:','Grunge -', Popular_Genre.count('Grunge'))
if Popular_Genre.count('Rock') > Popular_Genre.count('Alternative'):
    print('Most popular genre:','Rock -', Popular_Genre.count('Rock'))
else:
    print('Most popular genre:','Alternative -', Popular_Genre.count('Alternative'))

Most popular genre: Alternative - 24


In [105]:
release_date_extractor = lambda x: x['releaseDate']

Release_date = min(Album_value, key=release_date_extractor)
print('Release date:', Release_date['trackName'], Release_date['releaseDate'])

Release date: Smells Like Teen Spirit (Live at Reading, 08/30/92) 2009-11-03 08:00:00+00:00


In [106]:
Album_Song_cost = lambda x: x['trackPrice']

All_album_price = sum(list(map(Album_Song_cost, Album_value)))
print('Price of all songs:', All_album_price, '$')

Average_album_price = All_album_price / Count_Of_Album_Songs
print('Average price of songs:', Average_album_price, '$')

Median_album_cost = list(map(Album_Song_cost, Album_value))
import statistics
print('Median of songs cost:', statistics.median(Median_album_cost), '$')

Expensive_album_song = max(Album_value, key=Album_Song_cost)
print('Expensive song:', Expensive_album_song['trackName'], Expensive_album_song['trackPrice'],'$')

Cheap_album_song = min(Album_value, key=Album_Song_cost)
if Cheap_album_song['trackPrice'] == -1 : 
    print('Cheap song:', Cheap_album_song['trackName'],'Free')
else:
    print('Cheap song:', Cheap_album_song['trackName'], Cheap_album_song['trackPrice'], '$')

Price of all songs: 28.559999999999985 $
Average price of songs: 1.1899999999999993 $
Median of songs cost: 1.29 $
Expensive song: Smells Like Teen Spirit (Live at Reading, 08/30/92) 1.29 $
Cheap song: The Money Will Roll Right In (Live at Reading, 08/30/92) 0.99 $
